## Doing Augmentation

https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/

In [1]:
import os
import albumentations as alb
import cv2
import matplotlib.pyplot as plt

## Working perfect

In [88]:
augmentor_without_boudingbox = alb.Compose([alb.RandomCrop(width=450, height=450), 
            alb.HorizontalFlip(p=0.5), 
            alb.RandomBrightnessContrast(p=0.5),
            alb.RandomGamma(p=0.5), 
            alb.RGBShift(p=0.5), 
            alb.VerticalFlip(p=0.5)])
augmentor = alb.Compose([alb.RandomCrop(width=450, height=450), 
                        alb.HorizontalFlip(p=0.5), 
                        alb.RandomBrightnessContrast(p=0.5),
                        alb.RandomGamma(p=0.5), 
                        alb.RGBShift(p=0.5), 
                        alb.VerticalFlip(p=0.5)], 
                    bbox_params=alb.BboxParams(format='yolo', 
                                                label_fields=['class_labels']))

In [87]:
for partition in ['test']: 
    for image in os.listdir(os.path.join(partition, 'images')):
        img = cv2.imread(os.path.join(partition, 'images', image))

        coords = [0,0,0,0]
        label_path = os.path.join(partition, 'labels', f'{image.split(".")[0]}.txt')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                content = f.readline().strip("\n")
            label = [float(value) for value in content.split(" ")]
            coords[0] = label[1]
            coords[1] = label[2]
            coords[2] = label[3]
            coords[3] = label[4]
        for x in range(2):
            if coords != [0,0,0,0] or coords != [0.0,0.0,0.0,0.0]:
                augmented = augmentor(image=img, bboxes=[coords], class_labels=['face'])
            else:
                augmented = augmentor_without_boudingbox(image=img)
            directory_path = os.path.join('aug_data', partition, 'images')
            os.makedirs(directory_path, exist_ok=True)
            cv2.imwrite(os.path.join('aug_data', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

            annotation = [0,0,0,0,0]

            if os.path.exists(label_path):
                if len(augmented['bboxes']) == 0: 
                    annotation[1] = 0
                    annotation[2] = 0
                    annotation[3] = 0
                    annotation[4] = 0
                    annotation[0] = 0 
                else: 
                    annotation[1] = augmented['bboxes'][0][0]
                    annotation[2] = augmented['bboxes'][0][1]
                    annotation[3] = augmented['bboxes'][0][2]
                    annotation[4] = augmented['bboxes'][0][3]
                    annotation[0] = 1
            else: 
                    annotation[1] = 0
                    annotation[2] = 0
                    annotation[3] = 0
                    annotation[4] = 0
                    annotation[0] = 0 
            directory_path_labels = os.path.join('aug_data', partition, 'labels')
            os.makedirs(directory_path_labels,exist_ok=True)
            with open(os.path.join('aug_data', partition, 'labels', f'{image.split(".")[0]}.{x}.txt'), 'w') as f:
                f.write(' '.join(map(str, annotation)))


In [ ]:
loc_img = "dummy/test/images/dbed8339-f7fe-11ed-85ce-38d547b2c17a.0.jpg"
loc_annot = "dummy/test/labels/dbed8339-f7fe-11ed-85ce-38d547b2c17a.0.txt"
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = plt.imread(loc_img)
with open(loc_annot,'r') as f:
    value = f.readline().strip()
    value_list = value.split()
value_list = [float(val)for val in value_list]

coords = value_list[1:]

augmented = augmentor(image=img,bboxes=[coords],class_labels=['face'])

# Assuming augmented['image'] is the augmented image you want to draw on
augmented_img =augmented['image']

# Assuming augmented['bboxes'][0] contains the augmented bounding box coordinates
augmented_bbox = augmented['bboxes'][0]

# Original image size
original_width = 640
original_height = 480

# Augmented image size
augmented_width = 450
augmented_height = 450

# Calculate the scale factors for width and height
width_scale = augmented_width / original_width
height_scale = augmented_height / original_height

# Scale the augmented bounding box coordinates to match the augmented image size
x_center = augmented_bbox[0] * original_width  # x-coordinate of the box center
y_center = augmented_bbox[1] * original_height  # y-coordinate of the box center
box_width = augmented_bbox[2] * original_width  # width of the box
box_height = augmented_bbox[3] * original_height  # height of the box

# Calculate the top-left and bottom-right coordinates of the box
x_min = int((x_center - (box_width / 2)) * width_scale)
y_min = int((y_center - (box_height / 2)) * height_scale)
x_max = int((x_center + (box_width / 2)) * width_scale)
y_max = int((y_center + (box_height / 2)) * height_scale)

new_coords = [x_min,y_min,x_max,y_max]

# Draw the rectangle on the augmented image
color = (255, 0, 0)  # Blue color
thickness = 2  # Thickness of the rectangle border
cv2.rectangle(augmented_img, (x_min, y_min), (x_max, y_max), color, thickness)

# Display the augmented image with the rectangle
plt.imshow(augmented_img)
plt.show()

## Trying out

In [89]:
img = cv2.imread("9e9e02a0-f7fe-11ed-ab83-38d547b2c17a.jpg")

coords = [0,0,0,0]
label_path = "9e9e02a0-f7fe-11ed-ab83-38d547b2c17a.txt"
if os.path.exists(label_path):
    with open(label_path, 'r') as f:
        content = f.readline().strip("\n")
    label = [float(value) for value in content.split(" ")]
    coords[0] = label[1]
    coords[1] = label[2]
    coords[2] = label[3]
    coords[3] = label[4]
for x in range(2):
    if coords != [0,0,0,0] or coords != [0.0,0.0,0.0,0.0]:
        augmented = augmentor(image=img, bboxes=[coords], class_labels=['face'])
    else:
        augmented = augmentor_without_boudingbox(image=img)
    directory_path = os.path.join('dummy', partition, 'images')
    os.makedirs(directory_path, exist_ok=True)
    cv2.imwrite(os.path.join('dummy', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

    annotation = [0,0,0,0,0]

    if os.path.exists(label_path):
        if len(augmented['bboxes']) == 0: 
            annotation[1] = 0
            annotation[2] = 0
            annotation[3] = 0
            annotation[4] = 0
            annotation[0] = 0 
        else: 
            annotation[1] = augmented['bboxes'][0][0]
            annotation[2] = augmented['bboxes'][0][1]
            annotation[3] = augmented['bboxes'][0][2]
            annotation[4] = augmented['bboxes'][0][3]
            annotation[0] = 1
    else: 
            annotation[1] = 0
            annotation[2] = 0
            annotation[3] = 0
            annotation[4] = 0
            annotation[0] = 0 
    directory_path_labels = os.path.join('dummy', partition, 'labels')
    os.makedirs(directory_path_labels,exist_ok=True)
    with open(os.path.join('dummy', partition, 'labels', f'{image.split(".")[0]}.{x}.txt'), 'w') as f:
        f.write(' '.join(map(str, annotation)))

In [98]:
img = cv2.imread("Garden3.jpg")

coords_list = []
label_path = "Garden3.txt"
if os.path.exists(label_path):
    with open(label_path, 'r') as f:
        content = f.readlines()

    for line in content:
        label = [float(value) for value in line.strip().split(" ")]
        coords = label[1:5]
        coords_list.append(coords)

for x in range(2):
    if coords_list:
        augmented = augmentor(image=img, bboxes=coords_list, class_labels=['face'] * len(coords_list))
    else:
        augmented = augmentor_without_boudingbox(image=img)

    directory_path = os.path.join('dummy', partition, 'images')
    os.makedirs(directory_path, exist_ok=True)
    cv2.imwrite(
        os.path.join('dummy', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'),
        augmented['image']
    )

    annotation = []

    if len(augmented['bboxes']) == 0:
        annotation.append(0)
    else:
        for bbox in augmented['bboxes']:
            annotation.append(1)
            annotation.extend(bbox)

    directory_path_labels = os.path.join('dummy', partition, 'labels')
    os.makedirs(directory_path_labels, exist_ok=True)
    with open(
        os.path.join('dummy', partition, 'labels', f'{image.split(".")[0]}.{x}.txt'),
        'w'
    ) as f:
        f.write('\n'.join(' '.join(map(str, annotation[i:i+5])) for i in range(0, len(annotation), 5)))

In [ ]:
loc_img = "dummy/test/images/dbed8339-f7fe-11ed-85ce-38d547b2c17a.0.jpg"
loc_annot = "dummy/test/labels/dbed8339-f7fe-11ed-85ce-38d547b2c17a.0.txt"
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = plt.imread(loc_img)
with open(loc_annot,'r') as f:
    value = f.readline().strip()
    value_list = value.split()
value_list = [float(val)for val in value_list]

coords = value_list[1:]

augmented = augmentor(image=img,bboxes=[coords],class_labels=['face'])

# Assuming augmented['image'] is the augmented image you want to draw on
augmented_img =augmented['image']

# Assuming augmented['bboxes'][0] contains the augmented bounding box coordinates
augmented_bbox = augmented['bboxes'][0]

# Original image size
original_width = 640
original_height = 480

# Augmented image size
augmented_width = 450
augmented_height = 450

# Calculate the scale factors for width and height
width_scale = augmented_width / original_width
height_scale = augmented_height / original_height

# Scale the augmented bounding box coordinates to match the augmented image size
x_center = augmented_bbox[0] * original_width  # x-coordinate of the box center
y_center = augmented_bbox[1] * original_height  # y-coordinate of the box center
box_width = augmented_bbox[2] * original_width  # width of the box
box_height = augmented_bbox[3] * original_height  # height of the box

# Calculate the top-left and bottom-right coordinates of the box
x_min = int((x_center - (box_width / 2)) * width_scale)
y_min = int((y_center - (box_height / 2)) * height_scale)
x_max = int((x_center + (box_width / 2)) * width_scale)
y_max = int((y_center + (box_height / 2)) * height_scale)

new_coords = [x_min,y_min,x_max,y_max]

# Draw the rectangle on the augmented image
color = (255, 0, 0)  # Blue color
thickness = 2  # Thickness of the rectangle border
cv2.rectangle(augmented_img, (x_min, y_min), (x_max, y_max), color, thickness)

# Display the augmented image with the rectangle
plt.imshow(augmented_img)
plt.show()

## making suitable (Finalized Version)

In [1]:
import cv2
import albumentations as alb
import os

In [2]:
W = 1280
H = 720

In [3]:
augmentor_without_boudingbox = alb.Compose([
    # alb.RandomCrop(width=W, height=H), 
            alb.HorizontalFlip(p=0.5), 
            alb.RandomBrightnessContrast(p=0.5),
            alb.Rotate(limit=30, p=0.5),
            alb.Blur(p=0.2),
            alb.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, p=0.2),
            alb.RandomGamma(p=0.5), 
            alb.RGBShift(p=0.5), 
            alb.VerticalFlip(p=0.5)])
augmentor = alb.Compose([
    # alb.RandomCrop(width=W, height=H), 
                        alb.HorizontalFlip(p=0.5), 
                        alb.RandomBrightnessContrast(p=0.5),
                        alb.Rotate(limit=30, p=0.5),
                        alb.Blur(p=0.2),
                        alb.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, p=0.2),
                        alb.RandomGamma(p=0.5), 
                        alb.RGBShift(p=0.5), 
                        alb.VerticalFlip(p=0.5)], 
                    bbox_params=alb.BboxParams(format='yolo', 
                                                label_fields=['class_labels']))

In [12]:
for i in os.listdir("Test/images/"):
    print(i)
    img_dir = "Test/images/"+i
    label_dir = "Test/labels/"+i.split(".")[0] + ".txt"
    image = i # image name 
    img = cv2.imread(img_dir)
    coords_list = []
    class_value = []
    label_path = os.path.join(label_dir)
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            content = f.readlines()
        for line in content:
            label = [float(value) for value in line.strip().split(" ")]
            class_value.append(int(label[0]))
            coords = label[1:5]
            coords_list.append(coords)
    for x in range(12):
        if coords_list:
            augmented = augmentor(image=img, bboxes=coords_list, class_labels=['face'] * len(coords_list))
        else:
            augmented = augmentor_without_boudingbox(image=img)
        partition = "Traintesting" # don't change this 
        directory_path = os.path.join('dummy', partition, 'images')
        os.makedirs(directory_path, exist_ok=True)
        cv2.imwrite(os.path.join('dummy', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'),augmented['image'])
        annotation = []
        if os.path.exists(label_path):
            for i in range(len(coords_list)):
                if i < len(augmented['bboxes']):
                    annotation.append(class_value[i])
                    annotation.extend(augmented['bboxes'][i])
                else:
                    annotation.append(0)
                    annotation.extend([0, 0, 0, 0])
        else:
            annotation.append(0)
            annotation.append(0)
            annotation.append(0)
            annotation.append(0)
            annotation.append(0)

        directory_path_labels = os.path.join('dummy', partition, 'labels')
        os.makedirs(directory_path_labels, exist_ok=True)
        with open(os.path.join('dummy', partition, 'labels', f'{image.split(".")[0]}.{x}.txt'),'w') as f:
            f.write('\n'.join(' '.join(map(str, annotation[i:i+5])) for i in range(0, len(annotation), 5)))

b_1_9.jpg
b_2_9.jpg
b_3_9.jpg
b_4_9.jpg


In [4]:
for partition in ['Train(2)']: 
    for image in os.listdir(os.path.join(partition, 'images')):
        img = cv2.imread(os.path.join(partition, 'images', image))
        coords_list = []
        class_value = []
        label_path = os.path.join(partition, 'labels', f'{image.split(".")[0]}.txt')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                content = f.readlines()
            for line in content:
                label = [float(value) for value in line.strip().split(" ")]
                class_value.append(int(label[0]))
                coords = label[1:5]
                coords_list.append(coords)
        for x in range(2):
            if coords_list:
                augmented = augmentor(image=img, bboxes=coords_list, class_labels=['face'] * len(coords_list))
            else:
                augmented = augmentor_without_boudingbox(image=img)
            directory_path = os.path.join('dummy', partition, 'images')
            os.makedirs(directory_path, exist_ok=True)
            cv2.imwrite(os.path.join('dummy', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'),augmented['image'])
            annotation = []
            if os.path.exists(label_path):
                for i in range(len(coords_list)):
                    if i < len(augmented['bboxes']):
                        annotation.append(class_value[i])
                        annotation.extend(augmented['bboxes'][i])
                    else:
                        annotation.append(0)
                        annotation.extend([0, 0, 0, 0])
            else:
                annotation.append(0)
                annotation.append(0)
                annotation.append(0)
                annotation.append(0)
                annotation.append(0)
       
            directory_path_labels = os.path.join('dummy', partition, 'labels')
            os.makedirs(directory_path_labels, exist_ok=True)
            with open(os.path.join('dummy', partition, 'labels', f'{image.split(".")[0]}.{x}.txt'),'w') as f:
                f.write('\n'.join(' '.join(map(str, annotation[i:i+5])) for i in range(0, len(annotation), 5)))

In [62]:
import cv2
import random
import os

org_image = "Garden4.jpg"
org_label = "Garden4.txt"
split = "Valid"
image_chosen = random.sample(os.listdir(f"New_Augmented_Data/{split}/images/"),1)[0].replace(".jpg","")
# "New_Augmented_Image\train\images\a4_16.0.jpg"
image_path = f"New_Augmented_Data//{split}//images//{image_chosen}.jpg"
label_path = f"New_Augmented_Data//{split}//labels//{image_chosen}.txt"

image = cv2.imread(image_path)
# Resize image for display
scale_percent = 50  # Adjust the value as needed
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
image = cv2.resize(image, (width, height))

with open(label_path, 'r') as f:
    lines = f.readlines()

color_mapping = {
    0: (255, 0, 0),  # Class 0 will be displayed in blue (BGR format)
    1: (0, 255, 233),  # Class 1 will be displayed in green
    2: (0, 0, 255)   # Class 2 will be displayed in red
}

for line in lines:
    label = line.strip().split(" ")
    class_id = int(label[0])
    x_center = float(label[1]) * image.shape[1]
    y_center = float(label[2]) * image.shape[0]
    width = float(label[3]) * image.shape[1]
    height = float(label[4]) * image.shape[0]

    x_min = int(x_center - width / 2)
    y_min = int(y_center - height / 2)
    x_max = int(x_center + width / 2)
    y_max = int(y_center + height / 2)

    # Draw bounding box rectangle
    color = color_mapping.get(class_id, (0, 0, 0))
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)

    # Display class label
    class_label = f"{class_id}"
    cv2.putText(image, class_label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

# Display the image with bounding box annotations
cv2.imshow("Image with Bounding Boxes", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Rename Files do our desired names

In [ ]:
import os
def Rename_Files(Image_Dir_List,Label_Dir_List,New_Name):
    folder_images_path = Image_Dir_List
    folder_label_path = Label_Dir_List
    Give_Me_Name = New_Name

    for dir in folder_images_path:
        file_list = os.listdir(dir)

        for i, filename in enumerate(file_list):
            new_filename = f"{Give_Me_Name}{i+1}.jpg"
            old_filepath = os.path.join(dir, filename)
            new_filepath = os.path.join(dir, new_filename)
            os.rename(old_filepath, new_filepath)
            
            print(f"Renamed {filename} to {new_filename}")
    for dir in folder_label_path:
        file_list = os.listdir(dir)

        for i,filename in enumerate(file_list):
            new_filename = f"{Give_Me_Name}{i+1}.txt"
            old_filepath = os.path.join(dir,filename)
            new_filepath = os.path.join(dir,new_filename)
            os.rename(old_filepath,new_filepath)

            print(f"Renamed {filename} to {new_filename}")

## Train valid test split

In [14]:
a = 10
a*0.8,a*0.15,a-8-1

(8.0, 1.5, 1)

In [15]:
8+1

9

In [16]:
8+1+1

10

In [17]:
import os
import random
import shutil

def split_dataset(image_dir, label_dir, train_ratio,  valid_ratio):
    # Create directories for train, test, and validation sets
    train_dir = "train"
    valid_dir = "valid"
    test_dir = "test"
    
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)
    os.makedirs(valid_dir, exist_ok=True)

    # Get a list of all image file names with the .jpg extension in the image directory
    image_files = [f for f in os.listdir(image_dir)]

    # Shuffle the list of image files randomly
    random.shuffle(image_files)

    # Calculate the number of images for each split based on the ratios
    total_images = len(image_files)
    train_split = int(total_images * train_ratio)
    valid_split = int(total_images * valid_ratio)

    # Move images to the respective split directories
    for i, image_file in enumerate(image_files):
        if i < train_split:
            split_dir = train_dir
        elif i < train_split + valid_split:
            split_dir = valid_dir
        else:
            split_dir = test_dir

        # Copy the image file to the split directory
        image_src = os.path.join(image_dir, image_file)
        image_dst = os.path.join(split_dir, "images", image_file)
        os.makedirs(os.path.dirname(image_dst), exist_ok=True)
        shutil.copy(image_src, image_dst)

        # Get the corresponding label file
        label_prefix = os.path.splitext(image_file)[0]
        label_file = label_prefix + ".txt"

        # Copy the label file to the split directory
        label_src = os.path.join(label_dir, label_file)
        label_dst = os.path.join(split_dir, "labels", label_file)
        os.makedirs(os.path.dirname(label_dst), exist_ok=True)
        shutil.copy(label_src, label_dst)

    print("Dataset split completed successfully.")

# Specify the directories and split ratios
image_directory = "to_do_data/images"
label_directory = "to_do_data/labels"
train_ratio = 0.8
valid_ratio = 0.15

# Call the split_dataset function
split_dataset(image_directory, label_directory, train_ratio, valid_ratio)


Dataset split completed successfully.


In [3]:
a = ["sr/ram"]
for i in a:
    print(i.split("/")[1])

ram


In [3]:
import os

image_files = os.listdir("Test/images/")
text_files = []

for image_file in image_files:
    base_name, _ = os.path.splitext(image_file)
    text_file = base_name + ".txt"
    text_files.append(text_file)
    with open(text_file, 'w') as file:
        pass  # Creates an empty file

print("Empty text files created:", text_files)

Empty text files created: ['b_1_9.txt', 'b_2_9.txt', 'b_3_9.txt', 'b_4_9.txt']


In [48]:
from WackyWarper.config import albumentation_custom as alb_c
import os
import cv2,shutil

In [82]:
def Start_Augmentor(list_of_directory:list, header_folder_name:str,images_needed:int):
    '''
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    list_of_directory-> list of directory your images and labels are located 
    eg: ['train','valid','test'] or just only ['train']
    the folder structure needs to be like this.
    (YOU HAVE TO CREATE IT AND PUT THE IMAGES AND LABELS ACCORDING TO THIS STRUCTURE!):
    train->images
         ->labels
    valid->images
         ->labels
    test->images
        ->labels
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    header_folder-> this is what your folder is going to be called
    eg: "Augmented_Images" the structure would be like this after created
    (AUTOMATICALLY CREATED FOR YOU!)
        Augmented_Images
                    train
                        images
                            augmented_images......
                        labels
                            augmented_labels.....
                    and for valid and test
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    images_needed-> How many augmented_images do you need per image
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    '''
    print(">>Augmentation started<<")
    folder_name = header_folder_name
    for partition in list_of_directory:
        for image in os.listdir(os.path.join(partition, 'images')):
            img = cv2.imread(os.path.join(partition, 'images', image))
            coords_list = []
            class_value = []
            label_path = os.path.join(partition, 'labels', f'{image.split(".")[0]}.txt')
            
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    content = f.readlines()
                for line in content:
                    label = [float(value) for value in line.strip().split(" ")]
                    class_value.append(int(label[0]))
                    coords = label[1:5]
                    coords_list.append(coords)
             
            for x in range(images_needed):
                try:
                    
                    if coords_list:
                        augmented = alb_c.augmentor(image=img, bboxes=coords_list, class_labels=['face'] * len(coords_list))
                    else:
                        augmented = alb_c.augmentor_without_boudingbox(image=img)
                    partitionl = partition.split("/")[1] 
                    directory_path = os.path.join(folder_name, partitionl, 'images')
                    os.makedirs(directory_path, exist_ok=True)
                    # Copy the original image to the augmented folder
                    original_image_path = os.path.join(partitionl, 'images', image)
                    new_image_path = os.path.join(folder_name, partitionl, 'images', f'{image.split(".")[0]}.jpg')
                    shutil.copyfile(original_image_path, new_image_path)

                    directory_path_labels = os.path.join(folder_name, partitionl, 'labels')
                    os.makedirs(directory_path_labels, exist_ok=True)

                    # Move the original label file to the augmented label folder
                    if coords_list:
                        original_label_path = os.path.join(partitionl, 'labels', f'{image.split(".")[0]}.txt')
                        new_label_path = os.path.join(folder_name, partitionl, 'labels', f'{image.split(".")[0]}.txt')
                        shutil.copyfile(original_label_path, new_label_path)
                    else:
                        with open(f'{folder_name}\{partitionl}\labels\{image.split(".")[0]}.txt','w') as f:
                            f.write("0 0 0 0 0")

                    cv2.imwrite(os.path.join(folder_name, partitionl, 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])
                    
                    annotation = []
                    if os.path.exists(label_path):
                        for i in range(len(coords_list)):
                            if i < len(augmented['bboxes']):
                                annotation.append(class_value[i])
                                annotation.extend(augmented['bboxes'][i])
                            else:
                                annotation.append(0)
                                annotation.extend([0, 0, 0, 0])
                    else:
                        annotation.append(0)
                        annotation.append(0)
                        annotation.append(0)
                        annotation.append(0)
                        annotation.append(0)
                    if annotation == []:
                        annotation.append(0)
                        annotation.append(0)
                        annotation.append(0)
                        annotation.append(0)
                        annotation.append(0)

                    with open(os.path.join(folder_name, partitionl, 'labels', f'{image.split(".")[0]}.{x}.txt'), 'w') as f:
                        f.write('\n'.join(' '.join(map(str, annotation[i:i+5])) for i in range(0, len(annotation), 5)))
                except:
                    continue
    print(">>Augmentation Ended<<")

In [83]:
Start_Augmentor(["Splitted/Train"],"Checking", 1)

>>Augmentation started<<
>>Augmentation Ended<<


In [37]:
from WackyWarper import helper_functions

helper_functions.Visualize("Checking/train/images/20230616123855_c2.0.jpg","Checking/train/labels/20230616123855_c2.0.txt")